In [12]:
import os
os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"

import numpy as np
import cv2
from glob import glob
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split
from patchify import patchify
import tensorflow as tf
from tensorflow.keras.callbacks import ModelCheckpoint, CSVLogger, ReduceLROnPlateau, EarlyStopping

In [13]:
""" Hyperparameters """
hp = {}
hp["image_size"] = 256
hp["num_channels"] = 3
hp["patch_size"] = 16
hp["num_patches"] = (hp["image_size"]**2) // (hp["patch_size"]**2)
hp["flat_patches_shape"] = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])

hp["batch_size"] = 16
hp["lr"] = 1e-4
hp["num_epochs"] = 3
hp["num_classes"] = 10
hp["class_names"] = ['Bacterial_Spot',
 'Curl_Virus',
 'Early_Blight',
 'Healthy',
 'Late_Blight',
 'Leaf_Mold',
 'Mosaic_Virus',
 'Septoria_Leaf_Spot',
 'Spider_Mites',
 'Target_Spot']

# cannot handle this hyperparameter
# hp["num_layers"] = 12
# hp["hidden_dim"] = 768
# hp["mlp_dim"] = 3072
# hp["num_heads"] = 12
# hp["dropout_rate"] = 0.1

hp["num_layers"] = 6
hp["hidden_dim"] = 256
hp["mlp_dim"] = 1024
hp["num_heads"] = 6
hp["dropout_rate"] = 0.05



In [14]:
def create_dir(path):
    if not os.path.exists(path):
        os.makedirs(path)

In [15]:
def load_data(path, split=0.1):
    images = shuffle(glob(os.path.join(path, "*", "*.jpg")))

    split_size = int(len(images) * split)
    train_x, valid_x = train_test_split(images, test_size=split_size, random_state=42)
    train_x, test_x = train_test_split(train_x, test_size=split_size, random_state=42)

    return train_x, valid_x, test_x

In [16]:
def process_image_label(path):
    """ Reading images """
    path = path.decode()
    image = cv2.imread(path, cv2.IMREAD_COLOR)
    image = cv2.resize(image, (hp["image_size"], hp["image_size"]))
#     image = image/255.0
    

    """ Preprocessing to patches """
    patch_shape = (hp["patch_size"], hp["patch_size"], hp["num_channels"])
    patches = patchify(image, patch_shape, hp["patch_size"])
#     patches = np.reshape(patches, (hp["num_patches"], hp["patch_size"], hp["patch_size"], hp["num_channels"]))
#     for i in range(hp["num_patches"]):
#         cv2.imwrite(f"files/{i}.png", patches[i])
#     return

    patches = np.reshape(patches, hp["flat_patches_shape"])
    patches = patches.astype(np.float32)

    """ Label """
    class_name = os.path.basename(os.path.dirname(path))
    class_idx = hp["class_names"].index(class_name)
    class_idx = np.array(class_idx, dtype=np.int32)

    return patches, class_idx


In [17]:
def parse(path):
    patches, labels = tf.numpy_function(process_image_label, [path], [tf.float32, tf.int32])
    labels = tf.one_hot(labels, hp["num_classes"])

    patches.set_shape(hp["flat_patches_shape"])
    labels.set_shape(hp["num_classes"])

    return patches, labels

def tf_dataset(images, batch=32):
    ds = tf.data.Dataset.from_tensor_slices((images))
    ds = ds.map(parse).batch(batch).prefetch(8)
    return ds

In [18]:
""" Seeding """
np.random.seed(42)
tf.random.set_seed(42)

In [19]:
""" Directory for storing files """
create_dir("files")

In [20]:
""" Paths """
dataset_path = "./Dataset"
model_path = os.path.join("files", "model.keras")
csv_path = os.path.join("files", "log.csv")

In [21]:
 """ Dataset """
train_x, valid_x, test_x = load_data(dataset_path)
print(f"Train: {len(train_x)} - Valid: {len(valid_x)} - Test: {len(test_x)}")


Train: 6400 - Valid: 800 - Test: 800


In [22]:
# for checking the patches remove comment function
process_image_label(train_x[0])

AttributeError: 'str' object has no attribute 'decode'

In [23]:
train_ds = tf_dataset(train_x, batch=hp["batch_size"])
valid_ds = tf_dataset(valid_x, batch=hp["batch_size"])
# for x,y in train_ds:
#     print(x.shape,y.shape)
    

In [ ]:
"""Model Implementation"""

In [24]:
import tensorflow as tf
from tensorflow.keras.layers import *
from tensorflow.keras.models import Model

In [25]:
class ClassToken(Layer):
    def __init__(self):
        super().__init__()

    def build(self, input_shape):
        w_init = tf.random_normal_initializer()
        self.w = tf.Variable(
            initial_value = w_init(shape=(1, 1, input_shape[-1]), dtype=tf.float32),
            trainable = True
        )

    def call(self, inputs):
        batch_size = tf.shape(inputs)[0]
        hidden_dim = self.w.shape[-1]

        cls = tf.broadcast_to(self.w, [batch_size, 1, hidden_dim])
        cls = tf.cast(cls, dtype=inputs.dtype)
        return cls

In [26]:
def mlp(x, cf):
    x = Dense(hp["mlp_dim"], activation="gelu")(x)
    x = Dropout(hp["dropout_rate"])(x)
    x = Dense(hp["hidden_dim"])(x)
    x = Dropout(hp["dropout_rate"])(x)
    return x

In [27]:
def transformer_encoder(x, cf):
    skip_1 = x
    x = LayerNormalization()(x)
    x = MultiHeadAttention(
        num_heads=hp["num_heads"], key_dim=hp["hidden_dim"]
    )(x, x)
    x = Add()([x, skip_1])

    skip_2 = x
    x = LayerNormalization()(x)
    x = mlp(x, cf)
    x = Add()([x, skip_2])

    return x

In [28]:
def ViT(hp):
    """ Inputs """
    input_shape = (hp["num_patches"], hp["patch_size"]*hp["patch_size"]*hp["num_channels"])
    inputs = Input(input_shape)     ## (None, 256, 3072)
    

    """ Patch + Position Embeddings """
    patch_embed = Dense(hp["hidden_dim"])(inputs)   ## (None, 256, 768)

    positions = tf.range(start=0, limit=hp["num_patches"], delta=1)
    pos_embed = Embedding(input_dim=hp["num_patches"], output_dim=hp["hidden_dim"])(positions) ## (256, 768)
    embed = patch_embed + pos_embed ## (None, 256, 768)

    """ Adding Class Token """
    token = ClassToken()(embed)
    x = Concatenate(axis=1)([token, embed]) ## (None, 257, 768)

    for _ in range(hp["num_layers"]):
        x = transformer_encoder(x, hp)

    """ Classification Head """
    x = LayerNormalization()(x)     ## (None, 257, 768)
    x = x[:, 0, :]
    x = Dense(hp["num_classes"], activation="softmax")(x)

    model = Model(inputs, x)
    return model

In [29]:
model = ViT(hp)

In [30]:
model.summary()

Model: "functional_1"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃ Param # ┃ Connected to         ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 256, 768)  │       0 │ -                    │
│ (InputLayer)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense (Dense)       │ (None, 256, 256)  │ 196,864 │ input_layer[0][0]    │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add (Add)           │ (None, 256, 256)  │       0 │ dense[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ class_token         │ (None, 1, 256)    │       0 │ add[0][0]            │
│ (ClassToken)        │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ concatenate         │ (None, 257, 256)  │       0 │ class_token[0][0],   │
│ (Concatenate)       │                   │         │ add[0][0]            │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalization │ (None, 257, 256)  │     512 │ concatenate[0][0]    │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 257, 256)  │ 1,577,… │ layer_normalization… │
│ (MultiHeadAttentio… │                   │         │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_1 (Add)         │ (None, 257, 256)  │       0 │ multi_head_attentio… │
│                     │                   │         │ concatenate[0][0]    │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 257, 256)  │     512 │ add_1[0][0]          │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_1 (Dense)     │ (None, 257, 1024) │ 263,168 │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_1 (Dropout) │ (None, 257, 1024) │       0 │ dense_1[0][0]        │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_2 (Dense)     │ (None, 257, 256)  │ 262,400 │ dropout_1[0][0]      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dropout_2 (Dropout) │ (None, 257, 256)  │       0 │ dense_2[0][0]        │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_2 (Add)         │ (None, 257, 256)  │       0 │ dropout_2[0][0],     │
│                     │                   │         │ add_1[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 257, 256)  │     512 │ add_2[0][0]          │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ multi_head_attenti… │ (None, 257, 256)  │ 1,577,… │ layer_normalization… │
│ (MultiHeadAttentio… │                   │         │ layer_normalization… │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ add_3 (Add)         │ (None, 257, 256)  │       0 │ multi_head_attentio… │
│                     │                   │         │ add_2[0][0]          │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ layer_normalizatio… │ (None, 257, 256)  │     512 │ add_3[0][0]          │
│ (LayerNormalizatio… │                   │         │                      │
├─────────────────────┼───────────────────┼─────────┼──────────────────────┤
│ dense_3 (Dense)     │ (None, 257, 1024) │ 263,168 │ layer_normalization

 Total params: 12,825,866 (48.93 MB)

 Trainable params: 12,825,866 (48.93 MB)

 Non-trainable params: 0 (0.00 B)

In [31]:
 model.compile(
        loss="categorical_crossentropy",
        optimizer=tf.keras.optimizers.Adam(hp["lr"], clipvalue=1.0),
        metrics=["acc"]
    )

In [32]:
 callbacks = [
        ModelCheckpoint(model_path, monitor='val_loss', verbose=1, save_best_only=True, save_freq='epoch'),
        ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=10, min_lr=1e-10, verbose=1),
        CSVLogger(csv_path),
        EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=False),
    ]

In [33]:
model.fit(
        train_ds,
        epochs=hp["num_epochs"],
        validation_data=valid_ds,
        callbacks=callbacks
    )

Epoch 1/3
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - acc: 0.1971 - loss: 2.3883
Epoch 1: val_loss improved from inf to 0.98657, saving model to files\model.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 1243s 3s/step - acc: 0.1974 - loss: 2.3871 - val_acc: 0.6725 - val_loss: 0.9866 - learning_rate: 1.0000e-04
Epoch 2/3
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - acc: 0.6528 - loss: 0.9565
Epoch 2: val_loss improved from 0.98657 to 0.84710, saving model to files\model.keras
400/400 ━━━━━━━━━━━━━━━━━━━━ 1306s 3s/step - acc: 0.6529 - loss: 0.9563 - val_acc: 0.7050 - val_loss: 0.8471 - learning_rate: 1.0000e-04
Epoch 3/3
400/400 ━━━━━━━━━━━━━━━━━━━━ 0s 7s/step - acc: 0.7869 - loss: 0.5958
Epoch 3: val_loss did not improve from 0.84710
400/400 ━━━━━━━━━━━━━━━━━━━━ 2850s 7s/step - acc: 0.7870 - loss: 0.5956 - val_acc: 0.6875 - val_loss: 0.9738 - learning_rate: 1.0000e-04


In [34]:
valid_ds = tf_dataset(valid_x, batch=hp["batch_size"])

In [ ]:
# Test

In [35]:
test_ds = tf_dataset(test_x, batch=hp["batch_size"])

In [36]:
model.load_weights(model_path)
model.compile(
        loss=tf.keras.losses.CategoricalCrossentropy(from_logits=False),
        optimizer=tf.keras.optimizers.Adam(hp["lr"]),
        metrics=["acc"]
)

In [37]:
model.evaluate(test_ds)

50/50 ━━━━━━━━━━━━━━━━━━━━ 115s 2s/step - acc: 0.7351 - loss: 0.7547


[0.7694843411445618, 0.7350000143051147]

In [38]:
from sklearn.metrics import classification_report, confusion_matrix

In [39]:
# Assuming test_ds yields batches of (x, y) tuples
test_predictions = []
test_true_labels = []

for test_images, test_labels in test_ds:
    # Predict the labels for the test images
    predictions = model.predict(test_images)

    # Convert predictions to class labels (assuming one-hot encoding)
    predicted_labels = np.argmax(predictions, axis=1)

    # Convert true labels to class labels
    true_labels = np.argmax(test_labels.numpy(), axis=1)

    # Append to lists for the entire test set
    test_predictions.extend(predicted_labels)
    test_true_labels.extend(true_labels)

# Convert lists to NumPy arrays
test_predictions = np.array(test_predictions)
test_true_labels = np.array(test_true_labels)

# Calculate the classification report
report = classification_report(test_true_labels, test_predictions, zero_division=1)
print("Classification Report:\n", report)

# Calculate the confusion matrix
conf_matrix = confusion_matrix(test_true_labels, test_predictions)
print("\nConfusion Matrix:\n", conf_matrix)

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 988ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 